In [1]:
import sys
project_root = "d:/MachineLearning/federated_vae"
sys.path.append(project_root)

In [2]:
from collections import OrderedDict
from typing import List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import DataLoader

import flwr
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()

d:\Anaconda\envs\TMenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-12 00:52:45,132	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Training on cuda
Flower 1.19.0 / PyTorch 2.5.1+cu121


In [3]:
NUM_CLIENTS = 2
BATCH_SIZE = 256
NUM_EPOCHS = 100

from utils import get_all_vocab, split_data
vocab = get_all_vocab(["../../data/IMDB"])
datasets = split_data(dir = "../../data/IMDB", num_split=NUM_CLIENTS, vocab = vocab, batch_size= BATCH_SIZE)

train_size:  25000
test_size:  25000
vocab_size:  5000
average length: 94.966


parsing texts: 100%|██████████| 12500/12500 [00:00<00:00, 12565.78it/s]


In [2]:
from model.ETM import ETM
from trainer.basic_trainer import BasicTrainer

# net = ETM(len(vocab)).to(DEVICE)

# trainer = BasicTrainer(model = net, dataset = datasets[0], epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,
#                        log_interval=10)

# trainer.train()


ModuleNotFoundError: No module named 'model'

In [5]:
# res = trainer.get_top_words()
# print(res)

In [6]:
def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

In [7]:
from main.data.basic_dataset import RawDataset
class FlowerClient(NumPyClient):
  def __init__(self, net, dataset : RawDataset):
    self.net = net
    self.dataset = dataset
    self.trainer = BasicTrainer(net, dataset, epochs = 1, log_interval=10, device = DEVICE)

  # return the current local model parameters
  def get_parameters(self, config):
    return get_parameters(self.net)

  # receive global parameter, train, return updated model to server
  def fit(self, parameters, config):
    set_parameters(self.net, parameters)
    self.trainer.train()

    return get_parameters(self.net), len(self.dataset.train_texts), {}

  # receive global parameter, evaluate model from local's data, return the evaluation result
  def evaluate(self, parameters, config):
    set_parameters(self.net, parameters)
    loss, acc = -1, -1
    return float(loss), 0, {"accuracy":float(acc)}
  


test = FlowerClient(ETM(len(vocab)), datasets[0])

In [8]:
def client_fn(context: Context) -> Client:
    """Create a Flower client representing a single organization."""

    # Load model
    net = ETM(len(vocab)).to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data partition
    # Read the node_config to fetch data partition associated to this node
    partition_id = context.node_config["partition-id"]
    dataset = datasets[partition_id]

    # Create a single Flower client representing a single organization
    # FlowerClient is a subclass of NumPyClient, so we need to call .to_client()
    # to convert it to a subclass of `flwr.client.Client`
    return FlowerClient(net, dataset).to_client()


# Create the ClientApp
client = ClientApp(client_fn=client_fn)

In [ ]:
print(test.net.parameters)

<bound method Module.parameters of ETM(
  (encoder1): Sequential(
    (0): Linear(in_features=5000, out_features=800, bias=True)
    (1): ReLU()
    (2): Linear(in_features=800, out_features=800, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.0, inplace=False)
  )
  (fc21): Linear(in_features=800, out_features=50, bias=True)
  (fc22): Linear(in_features=800, out_features=50, bias=True)
)>


In [9]:
def server_fn(context: Context) -> ServerAppComponents:
    """Construct components that set the ServerApp behaviour.

    You can use the settings in `context.run_config` to parameterize the
    construction of all elements (e.g the strategy or the number of rounds)
    wrapped in the returned ServerAppComponents object.
    """

    # Configure the server for 5 rounds of training
    config = ServerConfig(num_rounds=5)
    # Create FedAvg strategy
    strategy = FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=10,
        min_evaluate_clients=5,
        min_available_clients=10,
    )
    return ServerAppComponents(strategy=strategy, config=config)


# Create the ServerApp
server = ServerApp(server_fn=server_fn)

In [10]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda":
    backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 1.0}}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

In [11]:
# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
    verbose_logging=True
)

DEBUG 2025-07-12 00:53:05,147:     Asyncio event loop already running.
DEBUG 2025-07-12 00:53:05,149:     Logger propagate set to False
DEBUG 2025-07-12 00:53:05,151:     Pre-registering run with id 5060386062207563297
DEBUG 2025-07-12 00:53:05,153:     Using InMemoryState
DEBUG 2025-07-12 00:53:05,154:     Using InMemoryState
INFO 2025-07-12 00:53:05,157:      Starting Flower ServerApp, config: num_rounds=5, no round_timeout
DEBUG 2025-07-12 00:53:05,160:     Using InMemoryState
INFO 2025-07-12 00:53:05,160:      
DEBUG 2025-07-12 00:53:05,162:     Registered 2 nodes
DEBUG 2025-07-12 00:53:05,163:     Supported backends: ['ray']
DEBUG 2025-07-12 00:53:05,163:     Initialising: RayBackend
INFO 2025-07-12 00:53:05,163:      [INIT]
DEBUG 2025-07-12 00:53:05,164:     Backend config: {'client_resources': {'num_cpus': 1, 'num_gpus': 1.0}, 'init_args': {}, 'actor': {'tensorflow': 0}}
INFO 2025-07-12 00:53:05,164:      Requesting initial parameters from one random client
2025-07-12 00:53:09,0

RuntimeError: Exception in ServerApp thread